In [7]:
import pandas as pd
import numpy as np

In [11]:
class utility_matrix:
    def __init__(self,path,ratings_column,product_id_column,user_id_column):
        self.df = pd.read_csv(path) # csv file as dataframe
        self.ratings_column = ratings_column # ratings column label
        self.product_id_column = product_id_column # product id column label
        self.user_id_column = user_id_column # user id column label
    def predict_ratings(self):
        user_item_ratings = pd.pivot_table(self.df,values=self.ratings_column,index=self.user_id_column,columns=self.product_id_column,aggfunc=np.min) #utility matrix
        user_item_ratings_binary = user_item_ratings.div(user_item_ratings).fillna(0) # utility matrix with values : 1 is user has rated item and 0 if not
        user_item_ratings = user_item_ratings.sub(user_item_ratings.mean(axis=1),axis=0) # normalize utility matrix
        item_item_correlations = user_item_ratings.corr() # item to item correlation matrix
        item_item_correlations = item_item_correlations.fillna(0)
        user_item_ratings = user_item_ratings.fillna(0)
        ratings_count = pd.DataFrame(self.df.groupby(self.product_id_column)[self.ratings_column].count()) # list of products with their total ratings count
        ratings_count = ratings_count.loc[(ratings_count!=0).any(axis=1)] # removing products that have 0 ratings
        sum__similarities_user_rated_items = user_item_ratings_binary.dot(item_item_correlations.abs()) # sum of similarities between user rated items and other items
        sum_ratings_count_per_user = user_item_ratings_binary.dot(ratings_count[self.ratings_column]) #list of total rating count of items that user has rated
        user_item_ratings.dot(item_item_correlations)
        predictions = (user_item_ratings.mul(ratings_count[self.ratings_column],axis=1)).dot(item_item_correlations).div(sum_ratings_count_per_user,axis='index').div(sum__similarities_user_rated_items)
        predictions = predictions.mul(user_item_ratings_binary.replace(to_replace = 1 ,value = np.nan).replace(to_replace = 0 ,value = 1)) # removing user rated items from dataframe(only predicted items)
        return predictions

In [12]:
u1 = utility_matrix(path='C://Users/Dimitri/Desktop/reduced_reviews.csv',ratings_column = 'rating',product_id_column='productid',user_id_column='username')

In [14]:
u1.predict_ratings().to_csv('predictions.csv')